# 通过 langchain使用 faiss

https://medium.com/operations-research-bit/build-your-own-semantic-search-web-app-with-sentence-transformers-and-faiss-c304e04ca3c9

## 导入需要的库

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from sentence_transformers import InputExample
import os
import pickle

## 准备数据

In [2]:
data = [['六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。', '工具'],
['烤白薯是一道美味健康的小吃，下面是简单的制作步骤', '菜谱'],
['老鼠是一种常见的小型哺乳动物，属于鼠科。', '动物'],
['比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg', '电商'],
['奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。', '食品'],
['凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g','电商'],
['黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。','天体物理'],
['"阿拉斯加的鳄鱼" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。','修辞'],
['潭中鱼可百许头，皆若空游无所依。','古代文学']]
df = pd.DataFrame(data, columns = ['文本', '分类'])
df.to_csv('output.csv', index=False)
display(df)

,文本,分类
0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具
1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱
2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物
3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商
4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品
5,凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g,电商
6,黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。,天体物理
7,"""阿拉斯加的鳄鱼"" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。",修辞
8,潭中鱼可百许头，皆若空游无所依。,古代文学


In [3]:
pdf = pd.read_csv('output.csv')
pdf["id"] = pdf.index
pdf_subset = pdf.head(5)

pdf_subset

,文本,分类,id
0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具,0
1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱,1
2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物,2
3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商,3
4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品,4


## 文本编码为向量

In [4]:
model=SentenceTransformer("/models/bge-large-zh-v1.5")
text = df['文本']

text

0               六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。
1                             烤白薯是一道美味健康的小吃，下面是简单的制作步骤
2                                 老鼠是一种常见的小型哺乳动物，属于鼠科。
3                              比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg
4              奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。
5                      凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g
6    黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。
7            "阿拉斯加的鳄鱼" 这个短语通常是一种幽默或谐音，用来形容一个不存在的事物或情况。
8                                     潭中鱼可百许头，皆若空游无所依。
Name: 文本, dtype: object

In [5]:
vectors = model.encode(text)

vectors

array([[ 0.02792062, -0.03714844, -0.01159204, ..., -0.01725687,
        -0.00189181, -0.01966484],
       [-0.0010843 , -0.04289858,  0.03477966, ...,  0.0101853 ,
         0.02182652, -0.00230858],
       [-0.00641313, -0.02665939,  0.01096631, ...,  0.01449827,
         0.02765457,  0.00925983],
       ...,
       [-0.0168227 , -0.00879555, -0.01485688, ...,  0.00648775,
         0.04386019,  0.00162739],
       [-0.04652883, -0.01336174, -0.00904606, ..., -0.00231134,
        -0.00262135,  0.03224757],
       [ 0.02496058,  0.03283865, -0.01557186, ..., -0.03958138,
        -0.03401512,  0.04865715]], dtype=float32)

## 生成向量和索引文件

In [6]:
dense_vectors_file = 'dense_vectors.pkl'
faiss_index_file = 'faiss_index.index'

faiss.normalize_L2(vectors)
vectors

array([[ 0.02792063, -0.03714845, -0.01159204, ..., -0.01725687,
        -0.00189181, -0.01966485],
       [-0.0010843 , -0.04289858,  0.03477966, ...,  0.0101853 ,
         0.02182652, -0.00230858],
       [-0.00641313, -0.02665939,  0.01096631, ...,  0.01449827,
         0.02765457,  0.00925983],
       ...,
       [-0.0168227 , -0.00879555, -0.01485688, ...,  0.00648775,
         0.04386019,  0.00162739],
       [-0.04652883, -0.01336174, -0.00904606, ..., -0.00231134,
        -0.00262135,  0.03224757],
       [ 0.02496058,  0.03283865, -0.01557186, ..., -0.03958138,
        -0.03401512,  0.04865715]], dtype=float32)

In [7]:
with open(dense_vectors_file, 'wb') as file:
    pickle.dump(vectors, file)